# House price prediction using Keras Sequential API

Importing data and splitting into train, validation and test datasets

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

x_train_full, x_test , y_train_full, y_test = train_test_split(housing.data, housing.target,random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full,y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)
x_train.shape

(11610, 8)

Model development and testing

In [9]:
import tensorflow as tf

tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=x_train.shape[1:])
norm_layer.adapt(x_train)
model = tf.keras.models.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer,metrics=["RootMeanSquaredError"])
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid))
mse_test, rmse_test = model.evaluate(x_test, y_test)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 1.2575 - loss: 1.6876 - val_RootMeanSquaredError: 0.6722 - val_loss: 0.4518
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6637 - loss: 0.4412 - val_RootMeanSquaredError: 0.6227 - val_loss: 0.3878
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6138 - loss: 0.3769 - val_RootMeanSquaredError: 0.6178 - val_loss: 0.3817
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5993 - loss: 0.3595 - val_RootMeanSquaredError: 0.6025 - val_loss: 0.3630
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5855 - loss: 0.3429 - val_RootMeanSquaredError: 0.6121 - val_loss: 0.3747
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5830 - loss: 0.3402 - val_RootMeanSquaredError: 0.5912 - val_loss: 0.3495
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5682 - los

In [11]:
y_pred = model.predict(x_test)
y_pred

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step


array([[0.513377 ],
       [1.1582758],
       [4.882418 ],
       ...,
       [1.5627021],
       [2.4245079],
       [4.745033 ]], dtype=float32)

Saving the model

In [14]:
model.save("my_keras_model.keras")

Loading the model

In [17]:
model_new = tf.keras.models.load_model("my_keras_model.keras")
y_pred = model_new.predict(x_test)
y_pred

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step


array([[0.513377 ],
       [1.1582758],
       [4.882418 ],
       ...,
       [1.5627021],
       [2.4245079],
       [4.745033 ]], dtype=float32)